In [6]:
import requests
import pandas as pd
from datetime import date

def fetch_json(url):
    return requests.get(url).json()

def get_team_stats(team_id):
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats?stats=season&group=hitting,pitching"
    data = fetch_json(url)

    stats = {
        "win_pct": 0.0,
        "team_era": 0.0,
        "team_ops": 0.0
    }

    for group in data.get("stats", []):
        splits = group["splits"][0]["stat"]
        if group["group"]["displayName"] == "hitting":
            stats["team_ops"] = float(splits.get("ops", 0.0))
        elif group["group"]["displayName"] == "pitching":
            stats["team_era"] = float(splits.get("era", 0.0))

    # Fetch standings to calculate win percentage
    standings_url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats?stats=standings"
    standings_data = fetch_json(standings_url)

    try:
        team_record = standings_data["stats"][0]["splits"][0]["stat"]
        wins = int(team_record["wins"])
        losses = int(team_record["losses"])
        stats["win_pct"] = round(wins / (wins + losses), 3)
    except (KeyError, IndexError, ZeroDivisionError):
        stats["win_pct"] = 0.0  # fallback

    return stats


def get_pitcher_era(pitcher_id):
    url = f"https://statsapi.mlb.com/api/v1/people/{pitcher_id}/stats?stats=season&group=pitching"
    data = fetch_json(url)

    try:
        era = float(data["stats"][0]["splits"][0]["stat"]["era"])
    except (IndexError, KeyError):
        era = 0.0
    return era

def get_todays_game_features():
    today = date.today().isoformat()
    schedule = fetch_json(f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}")
    
    if not schedule["dates"]:
        print(f"No games today: {today}")
        return pd.DataFrame()

    rows = []
    for game in schedule["dates"][0]["games"]:
        gamePk = game["gamePk"]
        home = game["teams"]["home"]["team"]
        away = game["teams"]["away"]["team"]

        game_data = fetch_json(f"https://statsapi.mlb.com/api/v1/game/{gamePk}/boxscore")

        try:
            home_pitcher_id = game_data["teams"]["home"]["players"][f'ID{game_data["teams"]["home"]["pitchers"][0]}']["person"]["id"]
            away_pitcher_id = game_data["teams"]["away"]["players"][f'ID{game_data["teams"]["away"]["pitchers"][0]}']["person"]["id"]
        except:
            home_pitcher_id = away_pitcher_id = None

        home_stats = get_team_stats(home["id"])
        away_stats = get_team_stats(away["id"])
        home_pitcher_era = get_pitcher_era(home_pitcher_id) if home_pitcher_id else 0.0
        away_pitcher_era = get_pitcher_era(away_pitcher_id) if away_pitcher_id else 0.0

        rows.append({
            "game_id": gamePk,
            "home_team": home["name"],
            "away_team": away["name"],
            "home_win_pct": home_stats["win_pct"],
            "away_win_pct": away_stats["win_pct"],
            "home_team_era": home_stats["team_era"],
            "away_team_era": away_stats["team_era"],
            "home_team_ops": home_stats["team_ops"],
            "away_team_ops": away_stats["team_ops"],
            "home_pitcher_era": home_pitcher_era,
            "away_pitcher_era": away_pitcher_era,
        })

    return pd.DataFrame(rows)

# Run and show results
df = get_todays_game_features()
print(df.to_string(index=False))


 game_id            home_team             away_team  home_win_pct  away_win_pct  home_team_era  away_team_era  home_team_ops  away_team_ops  home_pitcher_era  away_pitcher_era
  777172    Chicago White Sox     Toronto Blue Jays           0.0           0.0           4.17           4.15          0.642          0.737              1.60              2.65
  777175    Milwaukee Brewers   Los Angeles Dodgers           0.0           0.0           3.70           4.35          0.704          0.783              3.39              4.42
  777163 San Francisco Giants Philadelphia Phillies           0.0           0.0           3.41           3.70          0.685          0.733              4.84              4.44
  777182       Detroit Tigers        Tampa Bay Rays           0.0           0.0           3.42           3.79          0.751          0.730              0.00              0.00
  777181    Baltimore Orioles         New York Mets           0.0           0.0           4.94           3.56          0

In [25]:
import requests
import pandas as pd
from datetime import date

def fetch_json(url):
    return requests.get(url).json()

def parse_innings(ip_str):
    """Convert MLB innings-pitched string (e.g. "6.1") to float (6 + 1/3)."""
    if not ip_str:
        return 0.0
    parts = ip_str.split('.')
    innings = int(parts[0])
    if len(parts) > 1:
        try:
            innings += int(parts[1]) / 3
        except:
            pass
    return innings

def get_team_id_by_name(name):
    teams = fetch_json("https://statsapi.mlb.com/api/v1/teams?sportId=1")["teams"]
    for t in teams:
        if name.lower() in t["teamName"].lower() or name.lower() in t["name"].lower():
            return t["id"]
    raise ValueError(f"No MLB team found matching '{name}'")

def get_game_ops_era_whip(gamePk, side):
    box = fetch_json(f"https://statsapi.mlb.com/api/v1/game/{gamePk}/boxscore")["teams"][side]
    players = box["players"]

    # Batting for OPS
    agg = {"H":0, "AB":0, "BB":0, "HBP":0, "SF":0, "TB":0}
    for pid in box["batters"]:
        b = players[f"ID{pid}"]["stats"]["batting"]
        agg["H"]   += b.get("hits", 0)
        agg["AB"]  += b.get("atBats", 0)
        agg["BB"]  += b.get("baseOnBalls", 0)
        agg["HBP"] += b.get("hitByPitch", 0)
        agg["SF"]  += b.get("sacrificeFlies", 0)
        agg["TB"]  += b.get("totalBases", 0)

    denom = agg["AB"] + agg["BB"] + agg["HBP"] + agg["SF"]
    obp = (agg["H"] + agg["BB"] + agg["HBP"]) / denom if denom > 0 else 0.0
    slg = agg["TB"] / agg["AB"] if agg["AB"] > 0 else 0.0
    ops = round(obp + slg, 3)

    # Pitching for ERA & WHIP
    er_sum = ip_sum = hits_sum = bb_sum = 0
    for pid in box["pitchers"]:
        p = players[f"ID{pid}"]["stats"]["pitching"]
        er_sum   += p.get("earnedRuns", 0)
        ip_sum   += parse_innings(p.get("inningsPitched", "0.0"))
        hits_sum += p.get("hits", 0)
        bb_sum   += p.get("baseOnBalls", 0)

    era  = round(er_sum / ip_sum * 9, 3) if ip_sum else 0.0
    whip = round((hits_sum + bb_sum) / ip_sum, 3) if ip_sum else 0.0

    return ops, era, whip

def get_season_to_date_stats(team_id, season=None):
    """
    Pull every Regular-Season game from 'season' through today,
    compute OPS/ERA/WHIP/run differential for 'team_id',
    and add running home/away records.
    """
    if season is None:
        season = date.today().year

    today = date.today().isoformat()
    sched_url = (
        f"https://statsapi.mlb.com/api/v1/schedule"
        f"?sportId=1"
        f"&season={season}"
        f"&seasonType=2"              # Regular Season
        f"&teamId={team_id}"          # only this team’s games
        f"&startDate={season}-03-01"
        f"&endDate={today}"
    )
    sched = fetch_json(sched_url).get("dates", [])

    rows = []
    for day in sched:
        for g in day["games"]:
            if g["status"]["detailedState"] not in ["Final", "Completed Early"]:
                continue

            home = g["teams"]["home"]["team"]
            away = g["teams"]["away"]["team"]
            gamePk = g["gamePk"]

            # identify venue
            if team_id == home["id"]:
                venue, opp, ts, os_, side = "Home", away["name"], g["teams"]["home"]["score"], g["teams"]["away"]["score"], "home"
            else:
                venue, opp, ts, os_, side = "Away", home["name"], g["teams"]["away"]["score"], g["teams"]["home"]["score"], "away"

            result = "W" if ts > os_ else "L"
            ops, era, whip = get_game_ops_era_whip(gamePk, side)
            run_diff = ts - os_

            rows.append({
                "date":            day["date"],
                "venue":           venue,
                "opponent":        opp,
                "result":          result,
                "runs_scored":     ts,
                "runs_allowed":    os_,
                "run_differential": run_diff,
                "game_ops":        ops,
                "game_era":        era,
                "game_whip":       whip
            })

    # sort chronologically and add cumulative records
    rows = sorted(rows, key=lambda x: x["date"])
    home_wins = home_losses = away_wins = away_losses = 0
    for r in rows:
        if r["venue"] == "Home":
            if r["result"] == "W":
                home_wins += 1
            else:
                home_losses += 1
        else:  # Away
            if r["result"] == "W":
                away_wins += 1
            else:
                away_losses += 1

        r["home_record"] = f"{home_wins}-{home_losses}"
        r["away_record"] = f"{away_wins}-{away_losses}"

    return pd.DataFrame(rows)

if __name__ == "__main__":
    name = input("Enter (part of) MLB team name: ")
    tid  = get_team_id_by_name(name)
    df   = get_season_to_date_stats(tid)
    print(df.to_string(index=False))


      date venue             opponent result  runs_scored  runs_allowed  run_differential  game_ops  game_era  game_whip home_record away_record
2025-03-01  Home         Chicago Cubs      L            1            11               -10     0.608    11.000      2.222         0-1         0-0
2025-03-02  Away     San Diego Padres      W           10             4                 6     0.727     4.000      1.222         0-1         1-0
2025-03-03  Home        Texas Rangers      W            6             3                 3     0.833     3.000      1.000         1-1         1-0
2025-03-05  Home     Seattle Mariners      W            9             6                 3     0.969     6.000      1.222         2-1         1-0
2025-03-06  Away         Chicago Cubs      W            9             3                 6     0.921     3.000      0.889         2-1         2-0
2025-03-07  Home   Los Angeles Angels      W            8             2                 6     1.400     3.000      1.667         3

In [6]:
#!/usr/bin/env python3
import asyncio
import aiohttp
import pandas as pd
import time
from datetime import date
from typing import Dict, Any
import nest_asyncio
import ssl
import certifi

nest_asyncio.apply()

# prepare SSL context
ssl_ctx = ssl.create_default_context(cafile=certifi.where())

def parse_innings(ip_str: str) -> float:
    if not ip_str:
        return 0.0
    parts = ip_str.split('.')
    innings = int(parts[0])
    if len(parts) > 1:
        try:
            innings += int(parts[1]) / 3
        except ValueError:
            pass
    return innings

async def fetch_json(session: aiohttp.ClientSession, url: str) -> Dict[str, Any]:
    async with session.get(url) as resp:
        resp.raise_for_status()
        return await resp.json()

async def compute_stats_for_side(box: Dict[str, Any], side: str) -> tuple[float, float, float]:
    b = box[side]
    players = b["players"]

    # OPS
    agg = {"H":0, "AB":0, "BB":0, "HBP":0, "SF":0, "TB":0}
    for pid in b["batters"]:
        st = players[f"ID{pid}"]["stats"]["batting"]
        agg["H"]   += st.get("hits", 0)
        agg["AB"]  += st.get("atBats", 0)
        agg["BB"]  += st.get("baseOnBalls", 0)
        agg["HBP"] += st.get("hitByPitch", 0)
        agg["SF"]  += st.get("sacrificeFlies", 0)
        agg["TB"]  += st.get("totalBases", 0)
    denom = agg["AB"] + agg["BB"] + agg["HBP"] + agg["SF"]
    obp   = (agg["H"] + agg["BB"] + agg["HBP"]) / denom if denom > 0 else 0.0
    slg   = agg["TB"] / agg["AB"] if agg["AB"] > 0 else 0.0
    ops   = round(obp + slg, 3)

    # ERA & WHIP
    er_sum = ip_sum = hits_sum = bb_sum = 0.0
    for pid in b["pitchers"]:
        pst = players[f"ID{pid}"]["stats"]["pitching"]
        er_sum   += pst.get("earnedRuns", 0)
        ip_sum   += parse_innings(pst.get("inningsPitched", "0.0"))
        hits_sum += pst.get("hits", 0)
        bb_sum   += pst.get("baseOnBalls", 0)
    era  = round((er_sum / ip_sum) * 9, 3) if ip_sum else 0.0
    whip = round((hits_sum + bb_sum) / ip_sum, 3) if ip_sum else 0.0

    return ops, era, whip

async def fetch_box_and_build_row(session: aiohttp.ClientSession, game: Dict[str, Any]) -> Dict[str, Any]:
    pk    = game["pk"]
    day   = game["date"]
    home  = game["home"]
    away  = game["away"]

    box = (await fetch_json(
        session,
        f"https://statsapi.mlb.com/api/v1/game/{pk}/boxscore"
    ))["teams"]

    h_ops, h_era, h_whip = await compute_stats_for_side(box, "home")
    a_ops, a_era, a_whip = await compute_stats_for_side(box, "away")
    hs, as_ = home["score"], away["score"]

    return {
        "date":           day,
        "home_team_id":   home["team"]["id"],
        "home_team":      home["team"]["name"],
        "away_team_id":   away["team"]["id"],
        "away_team":      away["team"]["name"],
        "home_score":     hs,
        "away_score":     as_,
        "home_ops":       h_ops,
        "home_era":       h_era,
        "home_whip":      h_whip,
        "away_ops":       a_ops,
        "away_era":       a_era,
        "away_whip":      a_whip,
        "result_home":    1 if hs > as_ else 0,
        "result_away":    1 if as_ > hs else 0,
    }

async def get_all_games_stats(season: int | None = None) -> pd.DataFrame:
    if season is None:
        season = date.today().year
    start_date = f"{season}-03-01"
    end_date   = date.today().isoformat()
    sched_url  = (
        f"https://statsapi.mlb.com/api/v1/schedule"
        f"?sportId=1&season={season}&seasonType=2"
        f"&startDate={start_date}&endDate={end_date}"
    )

    connector = aiohttp.TCPConnector(limit=100, ssl=ssl_ctx)
    async with aiohttp.ClientSession(connector=connector) as session:
        sched = (await fetch_json(session, sched_url)).get("dates", [])
        games = []
        for day in sched:
            for g in day["games"]:
                if g["status"]["detailedState"] not in ("Final", "Completed Early"):
                    continue
                games.append({
                    "date": day["date"],
                    "pk":   g["gamePk"],
                    "home": g["teams"]["home"],
                    "away": g["teams"]["away"]
                })

        # fetch & build rows in parallel
        tasks = [fetch_box_and_build_row(session, g) for g in games]
        rows  = await asyncio.gather(*tasks)

    df = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)

    # 1) cumulative home/away win percentages
    home_rec = {}   # team_id -> (wins, losses)
    away_rec = {}
    home_win_pct = []
    away_win_pct = []

    for _, r in df.iterrows():
        hid = r["home_team_id"]
        aid = r["away_team_id"]

        hw, hl = home_rec.get(hid, (0, 0))
        aw, al = away_rec.get(aid, (0, 0))

        total_h = hw + hl
        total_a = aw + al
        home_win_pct.append(hw / total_h if total_h > 0 else 0.0)
        away_win_pct.append(aw / total_a if total_a > 0 else 0.0)

        if r["result_home"] == 1:
            hw += 1
        else:
            hl += 1
        if r["result_away"] == 1:
            aw += 1
        else:
            al += 1

        home_rec[hid] = (hw, hl)
        away_rec[aid] = (aw, al)

    df["home_win_pct"] = home_win_pct
    df["away_win_pct"] = away_win_pct

    # 2) cumulative averages of OPS/ERA/WHIP by location
    home_stats = {}   # team_id → {'ops', 'era', 'whip', 'count'}
    away_stats = {}
    home_avg_ops, home_avg_era, home_avg_whip = [], [], []
    away_avg_ops, away_avg_era, away_avg_whip = [], [], []

    for _, r in df.iterrows():
        hid = r["home_team_id"]
        h = home_stats.get(hid, {"ops":0.0,"era":0.0,"whip":0.0,"count":0})
        cnt = h["count"]
        home_avg_ops .append(h["ops"]/cnt if cnt else 0.0)
        home_avg_era .append(h["era"]/cnt if cnt else 0.0)
        home_avg_whip.append(h["whip"]/cnt if cnt else 0.0)
        h["ops"]   += r["home_ops"]
        h["era"]   += r["home_era"]
        h["whip"]  += r["home_whip"]
        h["count"] += 1
        home_stats[hid] = h

        aid = r["away_team_id"]
        a = away_stats.get(aid, {"ops":0.0,"era":0.0,"whip":0.0,"count":0})
        cnt2 = a["count"]
        away_avg_ops .append(a["ops"]/cnt2 if cnt2 else 0.0)
        away_avg_era .append(a["era"]/cnt2 if cnt2 else 0.0)
        away_avg_whip.append(a["whip"]/cnt2 if cnt2 else 0.0)
        a["ops"]   += r["away_ops"]
        a["era"]   += r["away_era"]
        a["whip"]  += r["away_whip"]
        a["count"] += 1
        away_stats[aid] = a

    df["home_avg_ops"]  = home_avg_ops
    df["home_avg_era"]  = home_avg_era
    df["home_avg_whip"] = home_avg_whip
    df["away_avg_ops"]  = away_avg_ops
    df["away_avg_era"]  = away_avg_era
    df["away_avg_whip"] = away_avg_whip

    # drop raw stats and ID columns
    df = df.drop(columns=[
        "home_team_id", "away_team_id",
        "home_ops", "home_era", "home_whip",
        "away_ops", "away_era", "away_whip"
    ])

    return df

def main():
    start = time.perf_counter()
    df = asyncio.run(get_all_games_stats())
    elapsed = time.perf_counter() - start
    print(f"Fetched and processed {len(df)} games in {elapsed:.1f}s")
    print(df.to_string(index=True))
    df.to_csv("mlb_games_stats.csv", index=False)

if __name__ == "__main__":
    main()


Fetched and processed 1869 games in 5.3s
            date                  home_team                  away_team  home_score  away_score  result_home  result_away  home_win_pct  away_win_pct  home_avg_ops  home_avg_era  home_avg_whip  away_avg_ops  away_avg_era  away_avg_whip
0     2025-03-01          Baltimore Orioles         Pittsburgh Pirates           2           5            0            1      0.000000      0.000000      0.000000      0.000000       0.000000      0.000000      0.000000       0.000000
1     2025-03-01         Kansas City Royals               Chicago Cubs           1          11            0            1      0.000000      0.000000      0.000000      0.000000       0.000000      0.000000      0.000000       0.000000
2     2025-03-01       Washington Nationals              Miami Marlins           7           0            1            0      0.000000      0.000000      0.000000      0.000000       0.000000      0.000000      0.000000       0.000000
3     2025-03-01   